# Study 1: Preprocessing

First of all, all necessary libraries will be imported.

In [1]:
import csv
import json
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

import string
import nltk
import pandas as pd
import numpy as np
from scipy import stats
import sys
import csv
from collections import Counter
import statsmodels.formula.api as sm

tokenizer = RegexpTokenizer(r'\w+')
my_stop = stopwords.words('dutch')
stemmer = SnowballStemmer('dutch')
from nltk.tokenize import word_tokenize
import re

import itertools
import pickle
import plotly.express as px

# STTM lib from Github
from gsdmm import MovieGroupProcess


# Opening all text files:

We start by opening our previous created dataframe. We do not have consent to share the data including the Text messages. For this reasons the analysis steps have been muted, but the taken steps can be tracked.

In [8]:
#buzz_df2 = pd.read_csv("Input/Text/buzz5.csv", engine = "python")
#buzz_df2["Sender"] = buzz_df2["Sender"].astype(str)

# Data Cleaning and Preprocessing

First of all empty text were removed from the text datafiles. They were  marked as NA and subsequently deleted..

In [9]:
#buzz_df2['Text'].replace('', np.nan, inplace=True)

Lets see how much data is left, once empty Texts are removed:

In [10]:
#buzz_df2 = buzz_df2[buzz_df2['Text'].notna()]
#print(len(buzz_df2))

The dataset contains a lot of duplicate texts (due to re-sharing of messages). In the current analysis we were only interested in unique text, only only unique messages were retained. This considerably narrows the dataset down.

In [12]:
#unique_buzz = buzz_df2.drop_duplicates(subset=['Text'])

#print(len(unique_buzz2))

Continuing with the preprocessing: All text data will be tokenized, so that we split single messages into single words.

In [14]:
#unique_buzz['tokens'] = unique_buzz['Text'].apply(tokenizer.tokenize)

Further cleaning steps:

In [15]:
# 1. If a character appears three times+ in a row (heeeeeey) reduce it to two
#unique_buzz['triples'] = unique_buzz['tokens'].apply(lambda x: [re.sub(r'(\w)\1+',r'\1\1', w) for w in x ])

# 2. Lowercase the text                                                              
#unique_buzz['low'] = unique_buzz['triples'].map(lambda x: list(map(str.lower, x)))

# 3. Remove Dutch stop-words
#unique_buzz['stop'] = unique_buzz['low'].apply(lambda x: [item for item in x if item not in my_stop])

# 4. Remove one-letter fragments
#unique_buzz['stop'] = unique_buzz['low'].map(lambda x: [y for y in x if len(y) > 1])

# 5. Remove punctuation
#unique_buzz['punctuation'] = unique_buzz['stop'].apply(lambda x: [re.sub("\'", "", w) for w in x])

# 6. Stemming of the single tokens
#unique_buzz['stem'] = unique_buzz['punctuation'].apply(lambda x: [stemmer.stem(y) for y in x])

# Convert the stems to a single string (later necessary for some analysis)
#unique_buzz["stem_string"] = [','.join(map(str, l)) for l in unique_buzz['stem']]

# Only keep Text which are send by the kids and not the ones which are sent by the mymovez team
#unique_buzz = unique_buzz[unique_buzz["Type"] != "mymovez"]


# Finding food in the dataset

Now we aim to filter out communication which is related to food and eating. For this first scroll to the end of the document and run the food-dictionaries...

In [25]:
with open("Input/Dictionaries/eten_en_drinken_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    eten_en_drinken3 = json.load(fp)
    
with open("Input/Dictionaries/foods_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    foods2 = json.load(fp)
    
with open("Input/Dictionaries/drinks_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    drinks2 = json.load(fp)
    
with open("Input/Dictionaries/diet_related_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    diet_related2 = json.load(fp)
    
with open("Input/Dictionaries/supermarkets_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    supermarkets2 = json.load(fp)
    
with open("Input/Dictionaries/internet_slang_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    internet_slang2 = json.load(fp)
    
with open("Input/Dictionaries/restaurants_cafes_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    restaurants_cafes2 = json.load(fp)
    
with open("Input/Dictionaries/meals_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    meals2 = json.load(fp)
    
with open("Input/Dictionaries/dutch_meals_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    dutch_meals2 = json.load(fp)
    
with open("Input/Dictionaries/food_brands_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    food_brands2 = json.load(fp)
    
with open("Input/Dictionaries/diets_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    diets2 = json.load(fp)

with open("Input/Dictionaries/embeddings_stem.json", "r", encoding = "utf-8", errors = "ignore") as fp:
    embeddings2 = json.load(fp)

Also we aimed to filter out some words with a double meaning in Dutch, that can refer to a food, but oftentimes do not.

In [26]:
fi = open('Input/Dictionaries/double_words.txt', "r")
double_words = fi.read()

We remove all word with possible double-meaning from the texts:

In [27]:
def doublemeaning(item):
    for words in item:
        if words in double_words:
            item.remove(words)

In [28]:
doublemeaning(eten_en_drinken3)
doublemeaning(foods2)
doublemeaning(drinks2)
doublemeaning(diet_related2)
doublemeaning(supermarkets2)
doublemeaning(internet_slang2)
doublemeaning(restaurants_cafes2)
doublemeaning(internet_slang2)
doublemeaning(diets2)
doublemeaning(meals2)
doublemeaning(dutch_meals2)
doublemeaning(food_brands2)
doublemeaning(embeddings2)

Using our dictionaries we build masks for each of them, to filter the matching conversations in our dataset which contain at least one word from one of the diet-related dictionaries:

In [16]:
#mask3 = unique_buzz.stem.apply(lambda x: any(item for item in eten_en_drinken3 if item in x))
#mask4 = unique_buzz.stem.apply(lambda x: any(item for item in embeddings2 if item in x))

#mask_food = unique_buzz.stem.apply(lambda x: any(item for item in foods2 if item in x))
#mask_drinks = unique_buzz.stem.apply(lambda x: any(item for item in drinks2 if item in x))
#mask_restaurants = unique_buzz.stem.apply(lambda x: any(item for item in restaurants_cafes2 if item in x))
#mask_supermarkets = unique_buzz.stem.apply(lambda x: any(item for item in supermarkets2 if item in x))
#mask_related = unique_buzz.stem.apply(lambda x: any(item for item in diet_related2 if item in x))
#mask_slang = unique_buzz.stem.apply(lambda x: any(item for item in internet_slang2 if item in x))
#mask_brands = unique_buzz.stem.apply(lambda x: any(item for item in food_brands2 if item in x))
#mask_diet = unique_buzz.stem.apply(lambda x: any(item for item in diets2 if item in x))
#mask_meals = unique_buzz.stem.apply(lambda x: any(item for item in meals2 if item in x))
#mask_Dmeals = unique_buzz.stem.apply(lambda x: any(item for item in dutch_meals2 if item in x))


#eat_buzz = unique_buzz[mask_food]
#drink_buzz = unique_buzz[mask_drinks]
#restaurant_buzz = unique_buzz[mask_restaurants]
#supermarket_buzz = unique_buzz[mask_supermarkets]
#related_buzz = unique_buzz[mask_related]
#slang_buzz = unique_buzz[mask_slang]
#brand_buzz = unique_buzz[mask_brands]
#diet_buzz = unique_buzz[mask_diet]
#meal_buzz = unique_buzz[mask_meals]
#dmeal_buzz = unique_buzz[mask_Dmeals]
#total_buzz = unique_buzz[mask3]

#lets filter and count the occuring words for each category
#total_buzz["foods"] = unique_buzz.stem.apply(lambda x: [item for item in foods2 if item in x])
#total_buzz["drinks"] = unique_buzz.stem.apply(lambda x: [item for item in drinks2 if item in x])
#total_buzz["restaurants"] = unique_buzz.stem.apply(lambda x: [item for item in restaurants_cafes2 if item in x])
#total_buzz["supermarkets"] = unique_buzz.stem.apply(lambda x: [item for item in supermarkets2 if item in x])
#total_buzz["related"] = unique_buzz.stem.apply(lambda x: [item for item in diet_related2 if item in x])
#total_buzz["slang"] = unique_buzz.stem.apply(lambda x: [item for item in internet_slang2 if item in x])
#total_buzz["brands"] = unique_buzz.stem.apply(lambda x: [item for item in food_brands2 if item in x])
#total_buzz["diet"] = unique_buzz.stem.apply(lambda x: [item for item in diets2 if item in x])
##total_buzz["meals"] = unique_buzz.stem.apply(lambda x: [item for item in meals2 if item in x])
#total_buzz["Dmeals"] = unique_buzz.stem.apply(lambda x: [item for item in dutch_meals2 if item in x])


We recode the media variable in the following way: 0 = Message contained no attached picture, 1 = message contained a non-diet-related picture, 2 = message contained a diet-related picture.

In [17]:
#total_buzz.Media[total_buzz.Media.str.startswith('Foodzz', na=False)] = 2
#total_buzz.Media = total_buzz.Media.replace(to_replace =('^[a-zA-Z]+'), value = 1, regex = True)
#total_buzz.Media = total_buzz.Media.replace(to_replace =('^[0-9]+\.'), value = 1, regex = True)

#total_buzz["Media"] = total_buzz.Media.fillna(0)
#NO Media: 0, Some media: 1, Food media: 2

#total_buzz["Media"] = total_buzz["Media"].apply(float)

We create dummy variables for the Media variable:

In [34]:
#total_buzz = pd.concat([total_buzz, pd.get_dummies(total_buzz['Media'], prefix='Media')], axis=1)

Lets have a look at the number of messages for each dictionary.

In [37]:
#print("Food items: ", len(unique_buzz[mask_food]))
#print("Drink items: ",len(unique_buzz[mask_drinks]))
#print("Restaurant items: ",len(unique_buzz[mask_restaurants]))
#print("Supermarket items: ",len(unique_buzz[mask_supermarkets]))
#print("Diet-related items: ",len(unique_buzz[mask_related]))
#print("Food slang items: ",len(unique_buzz[mask_slang]))
#print("Food brand items: ",len(unique_buzz[mask_brands]))
#print("Diet related items: ",len(unique_buzz[mask_diet]))
#print("Meal items: ",len(unique_buzz[mask_meals]))
#print("Dutch Meal items: ",len(unique_buzz[mask_Dmeals]))

Food items:  635
Drink items:  132
Restaurant items:  20
Supermarket items:  22
Diet-related items:  1274
Food slang items:  35
Food brand items:  26
Diet related items:  45
Meal items:  194
Dutch Meal items:  154


After getting insight into how the division is between different diet-related categories in the data, we will put them all together again. Subsequently we receive a dataframe with messages, that contain at least one diet-related word.

In [18]:
#food_buzz = total_buzz[mask3]

Great! the food_buzz dataframe now contains all messages which in contain at least one word related to food/eating/drinking etc.

In [45]:
print("We are left with", len(food_buzz), "sentences, that are about food, eating or drinking")

We are left with 2138 sentences, that are about food, eating or drinking


Now lets filter out the exact food_words in each message and append them to a seperate column in the datadrame.

In [21]:
#def get_words(text):
    #match = []
    #for word in eten_en_drinken3:
        #if re.search(r'\b' + word + r'\b', text):
            #match.append(re.findall(r'\b' + word + r'\b', text))
                        
        #else:
           # continue
    #return match

#food_buzz['words'] = food_buzz['stem_string'].apply(get_words)


# making a simple list out of list of lists
#food_buzz["words"] = food_buzz["words"].apply(lambda x: [item for sublist in x for item in sublist])
#counting items
#food_buzz['words_no'] = food_buzz['words'].apply(len)

In [ ]:
#food_buzz["word_string"] = [','.join(map(str, l)) for l in food_buzz['words']]

Nice! Lets save the results into single dataframes.

In [ ]:
#food_buzz.to_pickle('Input/Text/food_buzz.pkl')

In [ ]:
#eat_buzz.to_pickle('Input/Text/eat_buzz.pkl')
#drink_buzz.to_pickle('Input/Text/drink_buzz.pkl')
#restaurant_buzz.to_pickle('Input/Text/restaurant_buzz.pkl')
#supermarket_buzz.to_pickle('Input/Text/supermarket_buzz.pkl')
#related_buzz.to_pickle('Input/Text/related_buzz.pkl')
#slang_buzz.to_pickle('Input/Text/slang_buzz.pkl')
#brand_buzz.to_pickle('Input/Text/brand_buzz.pkl')
#diet_buzz.to_pickle('Input/Text/diet_buzz.pkl')
#meal_buzz.to_pickle('Input/Text/meal_buzz.pkl')
#dmeal_buzz.to_pickle('Input/Text/dmeal_buzz.pkl')
#unique_buzz.to_pickle('Input/Text/unique_buzz.pkl')